<a href="https://colab.research.google.com/github/amodsgit/AmodTheCoder/blob/main/T5based_recipe_model_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install necessary packages
!pip -q install transformers

In [2]:
# Step 2: Import necessary libraries
from transformers import FlaxAutoModelForSeq2SeqLM, AutoTokenizer
import requests
import json


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Step 3: Define the model and tokenizer
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

In [5]:
# Step 4: Define prefix and generation parameters
prefix = "items: "
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

In [6]:
# Step 5: Define special tokens and their mapping
special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}

In [7]:
# Step 6: Define the function to skip special tokens
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")
    return text

In [8]:
# Step 7: Define the function for postprocessing the output
def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

In [9]:
COMMON_INGREDIENTS = ['salt']  # Define common ingredients here
API_NINJAS_KEY = "ElNmsjpzj8EURGIKbAj90Q==4iG8q4Fd43uSDQEP"  # Replace with your api-ninjas key

def estimate_calories(ingredients):
    total_calories = 0
    for ingredient in ingredients:
        # Request nutritional information from the food database
        response = requests.get('https://api.api-ninjas.com/v1/nutrition?query={}'.format(ingredient),
                                headers={'X-Api-Key': API_NINJAS_KEY})
        if response.status_code == requests.codes.ok:
            data = json.loads(response.text)
            if data:
                for item in data:
                    calories = item['calories']
                    print(f"Calories in {ingredient}: {calories}")
                    total_calories += calories
            else:
                print(f"Couldn't find nutritional information for {ingredient}")
        else:
            print("Error:", response.status_code, response.text)
    return total_calories

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    # Concatenate user-provided ingredients with common ingredients
    inputs = [prefix + ', '.join(COMMON_INGREDIENTS + inp.split(', ')) for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )

    return generated_recipe

In [16]:
def generate_recipe_json(items):
    generated = generation_function(items)
    recipes = []
    for recipe in generated:
        recipe_dict = {}
        blocks = recipe.split('\n\n')
        for block in blocks:
            if block.startswith('title:'):
                recipe_dict['Title'] = block.replace('title:', '').strip().capitalize()
            elif block.startswith('ingredients:'):
                ingredients = block.replace('ingredients:', '').split('--')
                ingredients = [ingredient.strip() for ingredient in ingredients]
                # calculate calories
                total_calories = estimate_calories(ingredients)
                recipe_dict['Ingredients'] = ingredients
                recipe_dict['Calories Per Portion'] = total_calories
            elif block.startswith('directions:'):
                directions = block.replace('directions:', '').split('--')
                directions = [direction.strip().capitalize() for direction in directions]
                recipe_dict['Directions'] = directions
        if recipe_dict:
            recipes.append(recipe_dict)
    return json.dumps(recipes, indent=4)

In [17]:
items = [
    "chicken thigh, garlic, bell pepper, onion, cauliflower, pasta, milk"
]

In [20]:
print(generation_function(items))

[' title: chicken, noodle or rice pasta\n ingredients: salt to taste-- chicken thigh meat, cubed-- garlic powder to taste-- 1 bell pepper, chopped-- 1 onion, chopped-- 1 head cauliflower, cut into flowerets-- 1 pkg. bow tie pasta, cooked and drained-- 2 tbsp. evaporated skim milk\n directions: simmer chicken thigh meat and garlic powder in water for 5 minutes.-- remove chicken.-- in same pan, cook bell pepper, onion and cauliflower.-- when chicken is done, add sauteed vegetables and pasta.-- add milk and stir.-- serve immediately with grated cheese.']


In [18]:
recipes_json = generate_recipe_json(items)
print(recipes_json)

[]


In [19]:
def generate_recipe_json(items):
    generated = generation_function(items)
    recipes = []
    for recipe in generated:
        recipe_dict = {}
        blocks = recipe.split('\n\n')
        if len(blocks) >= 3:  # Assuming there are at least 3 blocks (title, ingredients, directions)
            # Title
            recipe_dict['Title'] = blocks[0].replace('title:', '').strip().capitalize()
            # Ingredients
            ingredients_block = blocks[1].replace('ingredients:', '')
            ingredients = ingredients_block.split('--')
            ingredients = [ingredient.strip() for ingredient in ingredients]
            # Calculate calories
            total_calories = estimate_calories(ingredients)
            recipe_dict['Ingredients'] = ingredients
            recipe_dict['Calories Per Portion'] = total_calories
            # Directions
            directions_block = blocks[2].replace('directions:', '')
            directions = directions_block.split('--')
            directions = [direction.strip().capitalize() for direction in directions]
            recipe_dict['Directions'] = directions
            recipes.append(recipe_dict)
    return json.dumps(recipes, indent=4)

recipes_json = generate_recipe_json(items)
print(recipes_json)


[]
